In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
map_size = 2 #arcminutes
pixel_size = 1/2 #arcseconds

pixel_number = map_size*(60/pixel_size)

x = np.linspace(-1, 1, int(pixel_number))
y = np.linspace(-1, 1, int(pixel_number))
X, Y = np.meshgrid(x, y)
R = np.sqrt(X**2 + Y**2)

In [21]:
p = np.arange(0, 100, 3)

In [22]:
p

array([ 0,  3,  6,  9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48,
       51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99])